In [1]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from torch import Tensor
from typing import List, Tuple
from torch.utils.data import TensorDataset, DataLoader, random_split
import sys
sys.path.append('/Users/germa/thesis/bachelor_project2023/src/models/lstm')
from xlm_roberta import XLMRoberta
sys.path.append('/Users/germa/thesis/bachelor_project2023/src/utils')
from chunker import Chunker

/Users/germa/anaconda3/envs/project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_1857/4227447418.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [39]:
class LSTMOnXLMRoberta(nn.Module):
    def __init__(self, lstm_hidden_size, num_lstm_layers, model_name = 'xlm-roberta-base', train_path = '../../../data/train/train.csv', train_size = 0.8, batch_size = 32, shuffle = True):
        super(LSTMOnXLMRoberta, self).__init__()
        self.xlmroberta_model = XLMRoberta(model_name)
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
        self.chunker = Chunker(self.tokenizer, 512)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=lstm_hidden_size,
                            num_layers=num_lstm_layers,
                            batch_first=True,
                            bidirectional=False)
        self.fc = nn.Linear(lstm_hidden_size * 2, 1)

        self.train_df, self.train_label = self.get_data(train_path)
    
    def get_data(self, path):
        train_df = pd.read_csv(path)
        train_short_df = train_df.head(2)
        return train_short_df, train_df["overall"]

    def chunk_data(self, df):
        texts1, texts2 = df["text1"], df["text2"]
        input_ids = []
        for i in range(len(texts1)):
            input_id_1 = self.chunker.chunk(texts1[i])
            input_id_2 = self.chunker.chunk(texts2[i])
            input_ids.append([input_id_1, input_id_2])           
        return input_ids
        
    def _manage_device(self) -> None:
        """
            Manage the device to run the model on
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.xlmroberta_model.to(self.device)
    
    def get_embeddings(self, input_ids):
        """
            Get the embeddings from the XLM-Roberta model
        """
        #input_ids = input_ids.to(self.device)
        outputs = self.xlmroberta_model.run(input_ids)
        return outputs
    
    def get_loss(self, outputs, labels):
        # Instantiate nn.MSELoss
        loss_function = nn.MSELoss()
        # Compute the loss
        return loss_function(outputs, labels)
    
     
    def train(self):
        self._manage_device()
        train_data = self.chunk_data(self.train_df)
        labels = self.train_label
        input_embeddings =self.get_embeddings(train_data)
        for i, row in enumerate(input_embeddings):
            lstm_output_row = []
            for text in row:
                lstm_out, _ = self.lstm(text)
                lstm_out_last = lstm_out[:, -1, :]  # Extract output of the last time step
                lstm_output_row.append(lstm_out_last)
            nn_input = torch.cat(lstm_output_row, dim=1)
            output = self.fc(nn_input)
            label = torch.tensor(labels[i], dtype=torch.float32)
            loss = self.get_loss(output, label)
            loss.backward()
            
                

                

In [40]:
lstmonxlmroberta = LSTMOnXLMRoberta(2, 1)
lstmonxlmroberta.train()


Token indices sequence length is longer than the specified maximum sequence length for this model (748 > 512). Running this sequence through the model will result in indexing errors
/Users/germa/anaconda3/envs/project/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
